# School Choice Analysis This project aims to analyze various schools in Linköping based on different criteria to assist in making informed decisions for selecting a school for åk 7 students.

# Packages

In [232]:
pip install folium geopy

Note: you may need to restart the kernel to use updated packages.


In [261]:
import pandas as pd
from tabulate import tabulate
import folium 
from folium.plugins import Search
from geopy.distance import geodesic

# Data Loading and Cleaning

In [263]:
# Load the data file
file_path = r'/personal_search/data/School_choise.xlsm'
df = pd.read_excel(file_path)

# Set display option to show all columns
pd.set_option('display.max_columns', None)

# Display the dataframe columns
print("The dataframe columns are:")
print(df.columns)

# Convert columns to numeric, errors='coerce' will convert non-numeric values to NaN
df['Andel (%) elever behöriga till yrkesprog.'] = pd.to_numeric(df['Andel (%) elever behöriga till yrkesprog.'], errors='coerce')
df['Andel (%) elever behöriga till estetiskt program'] = pd.to_numeric(df['Andel (%) elever behöriga till estetiskt program'], errors='coerce')
df['Andel (%) elever behöriga till Naturvetenskapligt och tekniskt program'] = pd.to_numeric(df['Andel (%) elever behöriga till Naturvetenskapligt och tekniskt program'], errors='coerce')

The dataframe columns are:
Index(['Skola', 'Typ av huvudman', 'Huvudman', 'Antal elever', 'elev_år',
       'Andel (%) elever behöriga till yrkesprog.',
       'Andel (%) elever behöriga till estetiskt program',
       'Andel (%) elever behöriga till Ekonomi-, humanistiska och samhällsvetenskaps- program',
       'Andel (%) elever behöriga till Naturvetenskapligt och tekniskt program',
       'Profil_Inriktning', 'Plats', 'Latitude', 'Longitude', 'Ovrigt',
       'Finns_Gymnasiumskolan', 'Link'],
      dtype='object')


# Calculate average scores

In [196]:
avg_yrkesprog = df['Andel (%) elever behöriga till yrkesprog.'].mean() 
avg_estetiskt = df['Andel (%) elever behöriga till estetiskt program'].mean() 
avg_nat_tech = df['Andel (%) elever behöriga till Naturvetenskapligt och tekniskt program'].mean() 

print(f"\nAverage score for 'Andel (%) elever behöriga till yrkesprog.': {avg_yrkesprog:.2f}") 
print(f"Average score for 'Andel (%) elever behöriga till estetiskt program': {avg_estetiskt:.2f}") 
print(f"Average score for 'Andel (%) elever behöriga till Naturvetenskapligt och tekniskt program': {avg_nat_tech:.2f}")


Average score for 'Andel (%) elever behöriga till yrkesprog.': 84.33
Average score for 'Andel (%) elever behöriga till estetiskt program': 81.08
Average score for 'Andel (%) elever behöriga till Naturvetenskapligt och tekniskt program': 75.94


# List schools above or at the average

In [198]:
# List schools above or at the average for 'Andel (%) elever behöriga till ...'
schools_above_avg_yrkesprog = df[df['Andel (%) elever behöriga till yrkesprog.'] >= avg_yrkesprog][['Skola', 'elev_år', 'Profil_Inriktning']]
schools_above_avg_estetiskt = df[df['Andel (%) elever behöriga till estetiskt program'] >= avg_estetiskt][['Skola', 'elev_år', 'Profil_Inriktning']]
schools_above_avg_nat_tech = df[df['Andel (%) elever behöriga till Naturvetenskapligt och tekniskt program'] >= avg_nat_tech][['Skola', 'elev_år', 'Profil_Inriktning']]

# Display the DataFrame using tabulate
print("Schools at or above average for 'Andel (%) elever behöriga till yrkesprog.':")
print(tabulate(schools_above_avg_yrkesprog, headers='keys', tablefmt='pretty'))

# Add a blank line 
print("\n")

print("Schools at or above average for 'Andel (%) elever behöriga till estetiskt program':")
print(tabulate(schools_above_avg_estetiskt, headers='keys', tablefmt='pretty'))

# Add a blank line 
print("\n")

print("Schools at or above average for 'Andel (%) elever behöriga till Naturvetenskapligt och tekniskt program':")
print(tabulate(schools_above_avg_nat_tech, headers='keys', tablefmt='pretty'))

Schools at or above average for 'Andel (%) elever behöriga till yrkesprog.':
+----+---------------------------+---------+------------------------------------------------------------------------------------+
|    |           Skola           | elev_år |                                 Profil_Inriktning                                  |
+----+---------------------------+---------+------------------------------------------------------------------------------------+
| 0  |        Arenaskolan        |   550   |                                       Idrott                                       |
| 1  |      Berzeliusskolan      |   750   |                                   Idrott, MA, NO                                   |
| 2  |      Björkö friskola      |   260   |                            Idrott, SO, Konst och Design                            |
| 6  |      Folkungaskolan       |   780   | Kör, Dans, Konst/Design, Framtidsinriktning, Musikproduktion, Hem/Konsumentkunskap |
| 7  | Intern

# List schools with specific profile

In [202]:
profile_keywords = ['språk', 'MA', 'NO', 'Framtidsinriktning']
schools_with_profiles = df[df['Profil_Inriktning'].str.contains('|'.join(profile_keywords), case=False, na=False)][['Skola', 'elev_år', 'Profil_Inriktning']]

print("\nSchools with specific profile (språk, MA, NO, Framtidsinriktning):")
print(tabulate(schools_with_profiles, headers='keys', tablefmt='pretty'))


Schools with specific profile (språk, MA, NO, Framtidsinriktning):
+---+---------------------------+---------+------------------------------------------------------------------------------------+
|   |           Skola           | elev_år |                                 Profil_Inriktning                                  |
+---+---------------------------+---------+------------------------------------------------------------------------------------+
| 1 |      Berzeliusskolan      |   750   |                                   Idrott, MA, NO                                   |
| 4 |       Ekholmsskolan       |   360   |                                  Fotboll, MA, NO                                   |
| 5 |  Elsa Brändströms skola   |   430   |                                       Språk                                        |
| 6 |      Folkungaskolan       |   780   | Kör, Dans, Konst/Design, Framtidsinriktning, Musikproduktion, Hem/Konsumentkunskap |
| 7 | Internat. Engelska Skol

# List schools with gymnasium

In [204]:
schools_with_gymnasium = df[df['Finns_Gymnasiumskolan'].str.lower() == 'ja'][['Skola', 'elev_år', 'Profil_Inriktning']]

print("\nSchools with Gymnasium Schools included:")
print(tabulate(schools_with_gymnasium, headers='keys', tablefmt='pretty'))


Schools with Gymnasium Schools included:
+---+---------------------------+---------+------------------------------------------------------------------------------------+
|   |           Skola           | elev_år |                                 Profil_Inriktning                                  |
+---+---------------------------+---------+------------------------------------------------------------------------------------+
| 1 |      Berzeliusskolan      |   750   |                                   Idrott, MA, NO                                   |
| 5 |  Elsa Brändströms skola   |   430   |                                       Språk                                        |
| 6 |      Folkungaskolan       |   780   | Kör, Dans, Konst/Design, Framtidsinriktning, Musikproduktion, Hem/Konsumentkunskap |
| 7 | Internat. Engelska Skolan |   643   |                         Språk, Internationalisering/språk                          |
| 8 |     JENSEN grundskola     |   139   |            

# Rank schools based on multiple conditions

In [220]:
# Assign points based on ranking
def assign_points(dataframe, num_points):
    points = {}
    for i, row in dataframe.iterrows():
        points[row['Skola']] = num_points - i
    return points

points_yrkesprog = assign_points(schools_above_avg_yrkesprog, 9)
points_estetiskt = assign_points(schools_above_avg_estetiskt, 9)
points_nat_tech = assign_points(schools_above_avg_nat_tech, 9)  # Correct to 9 points for consistency in ranking
points_profiles = assign_points(schools_with_profiles, 5)

# Sum up the points for each school
total_points = {}
for school in set(points_yrkesprog) | set(points_estetiskt) | set(points_nat_tech) | set(points_profiles):
    total_points[school] = (points_yrkesprog.get(school, 0) +
                            points_estetiskt.get(school, 0) +
                            points_nat_tech.get(school, 0) +
                            points_profiles.get(school, 0))

# Convert to DataFrame for sorting
total_points_df = pd.DataFrame(list(total_points.items()), columns=['Skola', 'Total Points'])
total_points_df = total_points_df.merge(df[['Skola', 'elev_år', 'Profil_Inriktning']], on='Skola')

# Sort the schools by total points
sorted_schools = total_points_df.sort_values(by='Total Points', ascending=False)

print("\nSchools sorted by total points, based on Profil_Inriktning and highest scores in behörigheten till Gymnasium:")
print(tabulate(sorted_schools, headers='keys', tablefmt='pretty'))


Schools sorted by total points, based on Profil_Inriktning and highest scores in behörigheten till Gymnasium:
+----+---------------------------+--------------+---------+------------------------------------------------------------------------------------+
|    |           Skola           | Total Points | elev_år |                                 Profil_Inriktning                                  |
+----+---------------------------+--------------+---------+------------------------------------------------------------------------------------+
| 3  |      Berzeliusskolan      |      28      |   750   |                                   Idrott, MA, NO                                   |
| 6  |        Arenaskolan        |      27      |   550   |                                       Idrott                                       |
| 10 |      Björkö friskola      |      21      |   260   |                            Idrott, SO, Konst och Design                            |
| 7  |      Folkung

# Distance from "Home" to school

In [267]:
# Convert sample data to DataFrame 
df = pd.DataFrame(data)

# Coordinates for 'Home'
home_coords = (58.40653, 15.56610)

# Create a map with satellite view
m = folium.Map(location=home_coords, zoom_start=13, tiles='OpenStreetMap')

# Add 'Home' marker
folium.Marker(
    location=home_coords,
    popup='Home',
    icon=folium.Icon(color='pink')
).add_to(m)

# Create a FeatureGroup for the schools to use with the Search plugin
school_markers = folium.FeatureGroup(name='Schools')

# Calculate distances and add school markers
for i, row in df.iterrows():
    school_coords = (row['Latitude'], row['Longitude'])
    distance = geodesic(home_coords, school_coords).km
    marker = folium.Marker(
        location=school_coords,
        popup=f"{row['Skola']}\nDistance: {distance:.2f} km",
        icon=folium.Icon(color='darkpurple')
    )
    marker.add_to(school_markers)

# Add the FeatureGroup with school markers to the map
school_markers.add_to(m)

# Add the Search plugin
search = Search(
    layer=school_markers,
    search_label='popup',
    placeholder='Search for a school',
    collapsed=False
)
search.add_to(m)

# Display the map
m